Exploring the data for Metro Nashville Emergency Communications project. Data provided through GitHub and consists of 4 .csv files that cover the tornado in March and the storm in May.
Notebook by Chris Mulvey
11 November 2020

In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

Reading in the .csv files and looking at their makeup.

In [ ]:
derecho_calls = pd.read_csv('../data/911_Phone Calls_Derecho_050320 050420.csv')

In [ ]:
derecho_calls.head()

In [ ]:
derecho_calls.info()

There are some null values in the derecho_calls file, in lat, long, and Cell Tower address columns.

In [ ]:
# Get a count of the number of each cell tower address for derecho_calls.
derecho_calls['Cell Tower Address'].value_counts()

In [ ]:
derecho_calls['Seizure DateTime'].value_counts()

In [ ]:
derecho_computer = pd.read_csv('../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv',
                               skiprows = 3) 
# Used skiprows because there is a textbox we do not need at the top of the csv file.

In [ ]:
derecho_computer.head()

In [ ]:
derecho_computer.info()

No null values in the derecho_computer file.

In [ ]:
# Get a count of the number of each incident type for derecho_computer.
derecho_computer['IncidentTypeDescription1'].value_counts()

In [ ]:
derecho_computer['IncidentDate'].value_counts()

In [ ]:
tornado_calls = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')

In [ ]:
tornado_calls.head()

In [ ]:
tornado_calls.info()

There are some nulls in the tornado_calls file, in the lat, long, and tower address columns.

In [ ]:
# Get a count of the number of each cell tower address for derecho_calls.
tornado_calls['Cell Tower Address'].value_counts()

In [ ]:
tornado_calls['Seizure DateTime'].value_counts()

In [ ]:
tornado_computer = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv', skiprows = 3)
# skiprows used again because of same type of text box as above.

In [ ]:
tornado_computer.head()

In [ ]:
tornado_computer.info()

No null values in tornado_computer.

In [ ]:
# Get a count of the number of each incident type for derecho_computer.
tornado_computer['IncidentTypeDescription1'].value_counts()

In [ ]:
tornado_computer['IncidentDate'].value_counts()

Seeing if I can search incident type for house or commercial, etc.

In [ ]:
tornado_houses = tornado_computer[tornado_computer['IncidentTypeDescription1'].str.contains('HOUSE')]

In [ ]:
tornado_houses.head()

Bringing in neighborhood maps to see which neighborhood the calls and dispatches were in.

In [ ]:
neighborhoods = gpd.read_file('../data/Neighborhood Association Boundaries (GIS).geojson')
print(neighborhoods.crs)

In [ ]:
neighborhoods.head()

Convert both DataFrames to GeoDataFrames by adding a geometry column.

In [ ]:
# Dropping rows with no lat and long
derecho_calls = derecho_calls.dropna(axis = 0, subset = ['ALI Longitude', 'ALI Latitude'])
# Making a geometry column in the derecho_calls DataFrame
derecho_calls['geometry'] = derecho_calls.apply(lambda x: Point((x['ALI Longitude']), 
                                                         (x['ALI Latitude'])), 
                                        axis=1)

In [ ]:
derecho_calls.head()

In [ ]:
# Matching up coordinate reference system between derecho_calls and neighborhoods.
derecho_calls_geo = gpd.GeoDataFrame(derecho_calls, 
                           crs = neighborhoods.crs, 
                           geometry = derecho_calls['geometry'])

derecho_calls_geo is ready to join with neighborhoods.

In [ ]:
# Same as derecho_call above.
tornado_calls = tornado_calls.dropna(axis = 0, subset = ['ALI Longitude', 'ALI Latitude'])
tornado_calls['geometry'] = tornado_calls.apply(lambda x: Point((x['ALI Longitude']), 
                                                         (x['ALI Latitude'])), 
                                        axis=1)

In [ ]:
tornado_calls.head(1)

In [ ]:
# Matching up coordinate reference system between tornado_calls and neighborhoods.
tornado_calls_geo = gpd.GeoDataFrame(tornado_calls, 
                           crs = neighborhoods.crs, 
                           geometry = tornado_calls['geometry'])

Moving on to the derecho and tornado computer aided dispatch DataFrames.

In [ ]:
derecho_computer = derecho_computer.dropna(axis = 0, subset = ['Longitude1', 'Latitude1'])
derecho_computer['geometry'] = derecho_computer.apply(lambda x: Point((x['Longitude1']), 
                                                         (x['Latitude1'])), 
                                        axis=1)

In [ ]:
derecho_computer.head(1)

In [ ]:
derecho_computer_geo = gpd.GeoDataFrame(derecho_computer, 
                           crs = neighborhoods.crs, 
                           geometry = derecho_computer['geometry'])

In [ ]:
tornado_computer = tornado_computer.dropna(axis = 0, subset = ['Longitude1', 'Latitude1'])
tornado_computer['geometry'] = tornado_computer.apply(lambda x: Point((x['Longitude1']), 
                                                         (x['Latitude1'])), 
                                        axis=1)

In [ ]:
tornado_computer.head(1)

In [ ]:
tornado_computer_geo = gpd.GeoDataFrame(tornado_computer, 
                           crs = neighborhoods.crs, 
                           geometry = tornado_computer['geometry'])

Joining neighborhoods with derecho_calls and derecho_computer.

In [ ]:
dcalls_by_neighborhood = gpd.sjoin(derecho_calls_geo, neighborhoods, op = 'within', how = 'left')

In [ ]:
#dcalls_by_neighborhood.dropna(axis = 0, subset = ['ALI Longitude', 'ALI Latitude'])

In [ ]:
dcalls_by_neighborhood.head()

In [ ]:
dcomp_by_neighborhood = gpd.sjoin(derecho_computer_geo, neighborhoods, op = 'within', how = 'left')

In [ ]:
dcomp_by_neighborhood.head()

In [ ]:
ax = neighborhoods.plot(figsize = (8, 10), color = 'lightgreen')
dcalls_by_neighborhood.plot( ax = ax);
plt.show();

In [ ]:
ax = neighborhoods.plot(figsize = (8, 10), color = 'lightgreen')
dcomp_by_neighborhood.plot( ax = ax);
plt.show();

In [ ]:
# setting a coordinate for Nashville city center. Got lat/long from google.
center = [36.1627, -86.7816]

In [ ]:
print(center)

In [ ]:
map_neighborhoods = folium.Map(location = center, zoom_start = 12)
map_neighborhoods

In [ ]:
#dcalls_by_neighborhood = dcalls_by_neighborhood.dropna()#axis = 0, subset = ['ALI Longitude', 'ALI Latitude'])
#dcomp_by_neighborhood = dcomp_by_neighborhood.dropna()#axis = 0, subset = ['Longitude1', 'Latitude1'])

map_neighborhoods = folium.Map(location =  center, zoom_start = 12)

folium.GeoJson(neighborhoods).add_to(map_neighborhoods)

# Plotting the derecho calls in red
for row_index, row_values in dcalls_by_neighborhood.iterrows():
    loc1 = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop1 = row_values['Seizure DateTime'], row_values['Cell Tower Address'], row_values['name']
    icon1 = folium.Icon(color = "red", icon = "phone-square", prefix = 'fa')
    marker1 = folium.Marker(
        location = loc1, 
        popup = pop1, icon = icon1)
    
    marker1.add_to(map_neighborhoods)
        
# Plotting the derecho dispatches in green    
for row_index, row_values in dcomp_by_neighborhood.iterrows():
    loc2 = [row_values['Latitude1'], row_values['Longitude1']]
    pop2 = row_values['IncidentDate'], row_values['IncidentTypeDescription1'], row_values['name']
    icon2 = folium.Icon(color = 'green', icon = 'phone', prefix = 'fa')
    marker2 = folium.Marker(
        location = loc2, 
        popup = pop2, icon = icon2)
    
    marker2.add_to(map_neighborhoods)
    
map_neighborhoods.save('../maps/mapdcalls.html')

map_neighborhoods

The above map has all dispatch rows plotted after call rows. The red marker is the a call that didn't have an associated dispatch.

I'll do the same with the tornado data.

In [ ]:
tcalls_by_neighborhood = gpd.sjoin(tornado_calls_geo, neighborhoods, op = 'within')

In [ ]:
tcalls_by_neighborhood.head()

In [ ]:
tcomp_by_neighborhood = gpd.sjoin(tornado_computer_geo, neighborhoods, op = 'within', how = 'left')

In [ ]:
tcomp_by_neighborhood.head()

In [ ]:
map_neighborhoodst = folium.Map(location =  center, zoom_start = 12)

folium.GeoJson(neighborhoods).add_to(map_neighborhoodst)

# Plotting the tornado calls in blue
for row_index, row_values in tcalls_by_neighborhood.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = row_values['Seizure DateTime'], row_values['Cell Tower Address'], row_values['name']
    icon = folium.Icon(color = 'blue', icon = 'phone-square', prefix = 'fa')
    marker1 = folium.Marker(
        location = loc, 
        popup = pop, icon = icon)
    
    marker1.add_to(map_neighborhoodst)
    
# Plotting the tornado dispatches in orange
for row_index, row_values in tcomp_by_neighborhood.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = row_values['IncidentDate'], row_values['IncidentTypeDescription1']#, row_values['name']
    icon = folium.Icon(color = 'orange', icon = 'phone', prefix = 'fa')
    marker2 = folium.Marker(
        location = loc, 
        popup = pop, icon = icon)
    
    marker2.add_to(map_neighborhoodst)
    
map_neighborhoodst.save('../maps/mapdcalls.html')

map_neighborhoodst